# Robot Aspirador SLAM con Batería Limitada 🔋
Este cuaderno simula un robot aspirador inteligente con SLAM al que se le añade batería limitada.

- Se le agota la batería con el movimiento.
- Si no tiene batería suficiente para llegar al objetivo, va a recargarse.
- Retoma el trabajo donde lo dejó tras la carga.

**Leyenda:**
- ⬜ Celda limpia
- 🟫 Celda sucia
- ⬛ Obstáculo
- 🤖 Robot
- 🔋 Estación de carga

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import time
from queue import Queue
from IPython.display import clear_output

# Parámetros
MAX_BATERIA = 20
TIEMPO_CARGA = 3

img_robot = plt.imread("roomba.png")
img_cargador = plt.imread("cargador.png")

def camino_disponible(grid, obstaculos, inicio, objetivo):
    q = Queue()
    q.put(inicio)
    visitados = set()
    while not q.empty():
        actual = q.get()
        if actual == objetivo:
            return True
        for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
            ny, nx = actual[0]+dy, actual[1]+dx
            if 0 <= ny < grid.shape[0] and 0 <= nx < grid.shape[1]:
                if obstaculos[ny, nx] == 0 and (ny, nx) not in visitados:
                    visitados.add((ny, nx))
                    q.put((ny, nx))
    return False

class EntornoSLAMBateria:
    def __init__(self, tamaño=None):
        while True:
            self.size = tamaño or random.choice([5, 6, 7])
            self.grid = np.random.choice([0, 1], size=(self.size, self.size), p=[0.3, 0.7])
            self.obstaculos = np.random.choice([0, 1], size=(self.size, self.size), p=[0.85, 0.15])
            self.obstaculos[0, 0] = 0
            self.obstaculos[-1, -1] = 0
            if camino_disponible(self.grid, self.obstaculos, (0,0), (self.size-1,self.size-1)):
                break
        self.robot_pos = (0, 0)
        self.carga_pos = (self.size-1, self.size-1)
        self.bateria = MAX_BATERIA

    def mostrar(self):
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(self.size, self.size))
        for i in range(self.size):
            for j in range(self.size):
                y_inv = self.size - 1 - i
                if (i, j) == self.robot_pos:
                    ax.imshow(img_robot, extent=(j, j+1, y_inv, y_inv+1), zorder=2)
                elif (i, j) == self.carga_pos:
                    ax.imshow(img_cargador, extent=(j, j+1, y_inv, y_inv+1), zorder=1)
                elif self.obstaculos[i, j] == 1:
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, color='black'))
                elif self.grid[i, j] == 1:
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, color='saddlebrown'))
                else:
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, edgecolor='gray', facecolor='white'))
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.axis('off')
        plt.title(f"Batería: {self.bateria}/{MAX_BATERIA}")
        plt.show()
        time.sleep(0.5)

In [ ]:
def vecinos_validos(pos, entorno):
    y, x = pos
    for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
        ny, nx = y+dy, x+dx
        if 0 <= ny < entorno.size and 0 <= nx < entorno.size:
            if entorno.obstaculos[ny, nx] == 0:
                yield (ny, nx)

def buscar_camino(entorno, inicio, objetivo):
    queue = Queue()
    queue.put((inicio, []))
    visitados = set()
    while not queue.empty():
        actual, camino = queue.get()
        if actual == objetivo:
            return camino + [actual]
        for vecino in vecinos_validos(actual, entorno):
            if vecino not in visitados:
                visitados.add(vecino)
                queue.put((vecino, camino + [actual]))
    return []

def cargar_robot(entorno):
    print("🔌 Cargando...")
    entorno.mostrar()
    time.sleep(TIEMPO_CARGA)
    entorno.bateria = MAX_BATERIA
    print("✅ Carga completa.")
    entorno.mostrar()

In [ ]:
def ejecutar_robot_bateria(entorno):
    pendientes = set((i,j) for i in range(entorno.size) for j in range(entorno.size)
                     if entorno.grid[i,j] == 1 and entorno.obstaculos[i,j] == 0)
    while pendientes:
        objetivos = sorted(pendientes, key=lambda pos: abs(pos[0]-entorno.robot_pos[0]) + abs(pos[1]-entorno.robot_pos[1]))
        destino = objetivos[0]
        camino = buscar_camino(entorno, entorno.robot_pos, destino)
        if entorno.bateria < len(camino):
            camino_carga = buscar_camino(entorno, entorno.robot_pos, entorno.carga_pos)
            for paso in camino_carga[1:]:
                entorno.robot_pos = paso
                entorno.bateria -= 1
                entorno.mostrar()
            cargar_robot(entorno)
        for paso in camino[1:]:
            entorno.robot_pos = paso
            entorno.bateria -= 1
            entorno.mostrar()
        entorno.grid[destino] = 0
        pendientes.remove(destino)
        entorno.mostrar()
    if entorno.robot_pos != entorno.carga_pos:
        camino = buscar_camino(entorno, entorno.robot_pos, entorno.carga_pos)
        for paso in camino[1:]:
            entorno.robot_pos = paso
            entorno.bateria -= 1
            entorno.mostrar()
        cargar_robot(entorno)
    print("✅ Limpieza finalizada y robot recargado.")

In [ ]:
entorno = EntornoSLAMBateria()
entorno.mostrar()
ejecutar_robot_bateria(entorno)